# Evaluating Model Performance Example

In [69]:
import seisbench
import seisbench.data as sbd
import seisbench.models as sbm
import seisbench.generate as sbg
import obspy
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch

import picts_ml.shared_data_files.shared_functions

ModuleNotFoundError: No module named 'shared_data_files'

Load the models to evaluate:

In [2]:
retrained_phasenet_stead = sbm.PhaseNet().load("trained_models/phasenet_stead_retrained_picts", version_str="1")
retrained_phasenet_ethz = sbm.PhaseNet().load("trained_models/phasenet_ethz_retrained_picts", version_str="1")
retrained_phasenet_instance = sbm.PhaseNet().load("trained_models/phasenet_instance_retrained_picts", version_str="1")

Load the test data. Note that it is important to ensure that none of the models have been trained using the test data.

In [37]:
csv_data = pd.read_csv("events_catalogue/preprocessed_event_data_2023-08-27 14:41:24.838809.csv", header=[0])

In [39]:
data.columns

Index(['Unnamed: 0', 'utc_datetime', 'lat', 'lon', 'depth', 'ML', 'RMS',
       'intensity', 'locality_1', 'locality_2', 'locality_3',
       'name_of_originating_file', 'comments',
       'PITL_p_arrival_time_man_picked', 'PITL_s_arrival_time_man_picked',
       'P3MM_p_arrival_time_man_picked', 'P3MM_s_arrival_time_man_picked',
       'P2LR_p_arrival_time_man_picked', 'P2LR_s_arrival_time_man_picked',
       'P1GW_p_arrival_time_man_picked', 'P1GW_s_arrival_time_man_picked',
       'G1WS_p_arrival_time_man_picked', 'G1WS_s_arrival_time_man_picked',
       'G2EB_p_arrival_time_man_picked', 'G2EB_s_arrival_time_man_picked',
       'G3GB_p_arrival_time_man_picked', 'G3GB_s_arrival_time_man_picked',
       'B1BC_p_arrival_time_man_picked', 'B1BC_s_arrival_time_man_picked',
       'DRUM_p_arrival_time_man_picked', 'DRUM_s_arrival_time_man_picked',
       'B3FZ_p_arrival_time_man_picked', 'B3FZ_s_arrival_time_man_picked',
       'TARL_p_arrival_time_man_picked', 'TARL_s_arrival_time_man_pi

In [40]:
trg_data = sbd.WaveformDataset("events_catalogue", sampling_rate=100, missing_componens="pad")

In [41]:
_, _, test = trg_data.train_dev_test()
test

In [43]:
test.metadata.columns

Index(['index', 'station_network_code', 'station_code', 'trace_channel',
       'station_latitude_deg', 'station_longitude_deg', 'station_elevation_m',
       'trace_p_arrival_sample', 'trace_p_status', 'trace_p_weight',
       'path_p_travel_sec', 'trace_s_arrival_sample', 'trace_s_status',
       'trace_s_weight', 'source_id', 'source_origin_time',
       'source_origin_uncertainty_sec', 'source_latitude_deg',
       'source_longitude_deg', 'source_error_sec', 'source_gap_deg',
       'source_horizontal_uncertainty_km', 'source_depth_km',
       'source_depth_uncertainty_km', 'source_magnitude',
       'source_magnitude_type', 'source_magnitude_author',
       'source_mechanism_strike_dip_rake', 'source_distance_deg',
       'source_distance_km', 'path_back_azimuth_deg', 'trace_snr_db',
       'trace_coda_end_sample', 'trace_start_time', 'trace_category',
       'trace_name', 'split', 'trace_name_original', 'trace_sampling_rate_hz',
       'trace_chunk', 'trace_component_order'],
   

In [65]:
p_arrivals = dict()
s_arrivals = dict()
for index in test.metadata.index:
    trace_start_time = obspy.UTCDateTime(test.metadata.loc[index, "trace_start_time"])
    sample_rate = int(test.metadata.loc[index, "trace_sampling_rate_hz"])
    p_arr_sample = int(test.metadata.loc[index, "trace_p_arrival_sample"])
    p_arr_time = trace_start_time + (p_arr_sample / sample_rate)
    s_arr_sample = int(test.metadata.loc[index, "trace_s_arrival_sample"])
    s_arr_time = trace_start_time + (s_arr_sample / sample_rate)
    station = test.metadata.loc[index, "station_code"]
    print(f"trace start: {trace_start_time}")
    print(f"arrival p: {p_arr_time}")
    print(f"arrival s: {s_arr_time}")

trace start: 2022-04-14T10:44:53.000000Z
arrival p: 2022-04-14T10:49:52.990000Z
arrival s: 2022-04-14T10:49:54.090000Z
trace start: 2022-04-19T02:58:40.590000Z
arrival p: 2022-04-19T03:03:40.580000Z
arrival s: 2022-04-19T03:03:42.450000Z
trace start: 2022-04-19T03:01:40.890000Z
arrival p: 2022-04-19T03:06:40.890000Z
arrival s: 2022-04-19T03:06:41.470000Z
trace start: 2022-04-19T11:21:48.500000Z
arrival p: 2022-04-19T11:26:48.490000Z
arrival s: 2022-04-19T11:26:49.120000Z
trace start: 2022-04-19T11:21:52.140000Z
arrival p: 2022-04-19T11:26:52.140000Z
arrival s: 2022-04-19T11:26:53.740000Z
trace start: 2022-04-20T11:16:06.440000Z
arrival p: 2022-04-20T11:21:06.440000Z
arrival s: 2022-04-20T11:21:08.290000Z
trace start: 2022-04-21T04:10:10.010000Z
arrival p: 2022-04-21T04:15:10.000000Z
arrival s: 2022-04-21T04:15:10.460000Z
trace start: 2022-04-23T18:34:30.490000Z
arrival p: 2022-04-23T18:39:30.480000Z
arrival s: 2022-04-23T18:39:31.170000Z
trace start: 2022-04-24T09:43:25.730000Z
arrival